# Avaliação Continuada 2 (AC2) - Árvore de Decisão

Abaixo, iremos criar um modelo para classificar mensagens de spam a partir de um dataset obtido no Kaggle.

Para esse problema, iremos usar o método de classificação Árvore de Decisão.

## Imports

Primeiro, começamos a importar todas as bibliotecas que iremos precisar nesse notebook, apenas para simplificar mais tarde.

In [66]:
# Leitura dos módulos
from __future__ import print_function

# biblioteca usada para trabalhar com dataframes
import pandas as pd
from pandas import Series,DataFrame

# uma biblioteca usada para auxiliar a limpeza de tags html
import html

# bibliotecas básicas como numpy e matplotlib para exibir 
# alguns gráficos caso seja necessário. 
import numpy as np
import matplotlib.pyplot as plt

# biblioteca para usar para medir acuracia e separar os dados de treino e teste
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# biblioteca usada para criar modelos de word embedding
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

np.random.seed(1000)

import re # biblioteca para expressoes regulares

import nltk # biblioteca para Processamento de Linguagem Natural
from nltk.stem.porter import PorterStemmer # para fazer a estemização em documentos da lingua inglesa
from nltk.stem import RSLPStemmer # para fazer a estemização em documentos da lingua portuguesa

Abaixo, um import que deve ser executado apenas um vez, caso não tenha executado, descomente a linha e execute novamente.

In [30]:
# nltk.download('all') # instala todos os recursos da biblioteca NLTK. Você pode descomentar esta linha na 1a vez que for executar este notebook

## Dataset

Depois, inicializamos o nosso dataset que foi baixado do Kaggle e pode ser obtido a partir [desse link](https://www.kaggle.com/team-ai/spam-text-message-classification).

In [31]:
data_set_url = "https://raw.githubusercontent.com/H4ad/h4ad.facens.artificial-inteligence.text-classification/master/datasets/spam-text-message-20170820.csv"
data_set = pd.read_csv(data_set_url,sep=",",header=0)

## Análise Exploratória

Abaixo, um pouco dos dados que iremos ver e tratar durante o notebook.

### O que há dentro?

Vamos começar primeiro a visualizar o que há dentro desse dataset com alguns comandos que nos mostram alguns informações básicas.

In [216]:
display(data_set.head(n=5))
display(data_set.describe())

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


### Valores Duplicados

Na tabela acima, podemos ver que na coluna "Message", temos um total de 5572 valores mas apenas 5157 são únicos.

Para não influenciar ou causar algum problema durante os treinos, rodar um outro comando para realmente ter a contagem de quantos dados duplicados nós temos:

In [33]:
data_set.duplicated().sum()

415

O número é de 415, é bem expressivo. Dessa forma, vamos agora tratar de apagar esses valores duplicados.

In [34]:
print('Shape antes de remover os dados.')
print(data_set.shape)

data_set_cleaned = data_set.drop_duplicates(inplace=False)

print('\nShape após remover os dados.')
print(data_set_cleaned.shape)

print('\nQuantidade de itens removidos:', (data_set.shape[0] - data_set_cleaned.shape[0]))

Shape antes de remover os dados.
(5572, 2)

Shape após remover os dados.
(5157, 2)

Quantidade de itens removidos: 415


### Valores Faltando

Após checar valores duplicados, podemos também verificar se não há algum valor faltando, usando o comando abaixo:

In [35]:
data_set_cleaned.isna().sum() / data_set_cleaned.shape[0]

Category    0.0
Message     0.0
dtype: float64

E como podemos ver, tudo zero para ambas as colunas, dessa forma, não precisamos fazer mais nada já que não há valores nulos ou faltando.

## Pré-Processamento

Após essa analise inicial dos dados, vamos começar com a etapa de pré-processamento dos textos obtidos.

As técnicas e o código abaixo usado foi obtido e modificado a partir do código original escrito pelo professor Renato Moraes Silva que pode ser encontrado [nesse notebook.](https://facens.instructure.com/courses/7531/files/1356091?module_item_id=269232)


Sobre as técnicas, serão usadas as seguintes:

 - deixar todas as palavras com letras minúsculas
 - substituir os números pela tag *\<NUMBER\>*
 - substituir todas as URLS pela tag *\<URL\>*
 - substituir todos os emails pela tag *\<EMAIL\>*
 - substituir o símbolo de moeda pela tag *\<MONEY\>*
 - substituir todos os caracteres não-alfanuméricos por um espaço em branco

E o que foi adicionado além das técnicas acima foram:
 - remover as entidades do html (`&gt;` `&#62;`).

Além disso, será aplicado também um processo de *estemização*, no qual diminui uma palavra para o seu radical, tornando uma palavra como "flies" para "fli". 

E também, será aplicado um processo de remoção das *stopwords*, no qual remove palavras muito comuns de uma lingua. Como estamos usando um dataset em inglês, palavras como "i", "me", "my", "myself", "we" e muitas outras.

> Uma lista de exemplos mais detalhadas pode ser encontrado [nesse gist](https://gist.github.com/sebleier/554280).

In [41]:
def preprocessing(text, stemming = False, stopwords = False):
    """
    Funcao usada para fazer o tratamento de textos da lingua inglesa
    
    Parametros: 
        text: variavel do tipo string que contem o texto que devera ser tratado
        
        stemming: variavel do tipo booleana que indica se a estemizacao deve ser aplicada ou nao
        
        stopwords: variavel do tipo booleana que indica se as stopwords devem ser removidas ou nao
    """
    
    # Lower case
    text = text.lower()
    
    # remove tags HTML
    regex = re.compile('<[^<>]+>')
    text = re.sub(regex, " ", text) 
    
    # remove as entidades do HTML
    regex = re.compile('(&.+;)')
    text = re.sub(regex, " ", text) 
    
    # normaliza as URLs
    regex = re.compile('(http|https)://[^\s]*')
    text = re.sub(regex, "<URL>", text)

    # normaliza emails
    regex = re.compile('[^\s]+@[^\s]+')
    text = re.sub(regex, "<EMAIL>", text)
    
    #normaliza o símbolo de dólar
    regex = re.compile('[$]+')
    text = re.sub(regex, "<MONEY>", text)
    
    # converte todos os caracteres não-alfanuméricos em espaço
    regex = re.compile('[^A-Za-z0-9]+')  
    text = re.sub(regex, " ", text)
    
    # normaliza os numeros 
    regex = re.compile('[0-9]+')
    text = re.sub(regex, "<NUMBER>", text)
    
    # substitui varios espaçamentos seguidos em um só
    text = ' '.join(text.split())

    # remove stopwords
    if stopwords:
        words = text.split() # separa o texto em palavras
        words = [w for w in words if not w in nltk.corpus.stopwords.words('english')]
        text = " ".join( words )
    
    # aplica estemização
    if stemming: 
        stemmer_method = PorterStemmer()  
        words = text.split() # separa o texto em palavras
        words = [ stemmer_method.stem(w) for w in words ]
        text = " ".join( words )
    
    # remove palavras de apenas um caracter
    words = text.split() # separa o texto em palavras
    words = [ w for w in words if len(w)>1 ]
    text = " ".join( words )
    
    return text

Com a função acima, podemos agora chamar ela para realizar todo o pré-processamento do nosso dataset.

In [44]:
print("Um dos textos antes do processamento:")
display(data_set_cleaned["Message"][201])

data_set_processed = data_set_cleaned.copy()
data_set_processed["Message"] = data_set_processed["Message"].apply(preprocessing)

print("O mesmo texto, só que agora processado:")
display(data_set_processed["Message"][201])

Um dos textos antes do processamento:


'I sent you  &lt;#&gt;  bucks'

O mesmo texto, só que agora processado:


'sent you bucks'

## Word Embedding (Transformando texto em número)

Após realizar o pré-processamento, temos todos os nossos textos e palavras bem organizadas e formatadas. Dessa forma, o que devemos fazer é transformar essas palavras em um formato que o nosso modelo de decisão em arvore irá conseguir entender, e é ai que entra o Word Embedding.

Word Embedding é uma técnica para transformar um texto em um vetor de atributos com valores numericos. Há alguns pré-treinados, como o da Google, mas iremos criar um próprio porque os pré-treinados são gigantescos (na ordem de Gigas) e serão muito potentes para um simples experimento.

Assim, iremos começar agora a criar uma lista de lista com as palavras de cada mensagem no dataset.

In [46]:
sentences = data_set_processed["Message"].apply(lambda message: message.split()).tolist()

print(sentences)

[['go',
  'until',
  'jurong',
  'point',
  'crazy',
  'available',
  'only',
  'in',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'there',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'in',
  '<NUMBER>',
  'wkly',
  'comp',
  'to',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  '<NUMBER>st',
  'may',
  '<NUMBER>',
  'text',
  'fa',
  'to',
  '<NUMBER>',
  'to',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply',
  '<NUMBER>over<NUMBER>'],
 ['dun', 'say', 'so', 'early', 'hor', 'already', 'then', 'say'],
 ['nah',
  'don',
  'think',
  'he',
  'goes',
  'to',
  'usf',
  'he',
  'lives',
  'around',
  'here',
  'though'],
 ['freemsg',
  'hey',
  'there',
  'darling',
  'it',
  'been',
  '<NUMBER>',
  'week',
  'now',
  'and',
  'no',
  'word',
  'back',
  'like',
  'some',
  'fun',
  'you',
  'up',
  'for',
  'it',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'to',
  'send',
  '<NUMB

E com essa lista de palavras, iremos agora treinar o modelo de Word Embedding utilizando a biblioteca [Genshim](https://radimrehurek.com/gensim/models/word2vec.html).

In [58]:
model = Word2Vec(sentences=sentences, vector_size=200, window=3, min_count=2, workers=4)

Com o modelo treinado, podemos ver qual é o tamanho do vocabulário da seguinte forma:

In [59]:
print('Tamanho do Vocabulario: ', + len(model.wv))

Tamanho do Vocabulario:  3632


E se precisarmos obter o valor associado a cada palavra, podemos usar o seguinte código:

In [61]:
print(model.wv["call"])

array([-0.16333361,  0.4818723 ,  0.07042091,  0.16689548,  0.18017298,
       -0.80066407, -0.01121409,  1.1005563 , -0.3210082 , -0.31221327,
       -0.07759333, -0.6131272 , -0.03102677,  0.35075384,  0.22945182,
       -0.34295484,  0.1972079 , -0.35620373, -0.08689868, -0.95462364,
        0.18158814,  0.25317657,  0.06905085, -0.16762389, -0.0542661 ,
       -0.06562439, -0.33037606, -0.20781477, -0.45240733,  0.05267035,
        0.5319103 , -0.1472507 ,  0.31292647, -0.31951636, -0.28743234,
        0.54570466,  0.18251696, -0.48515117, -0.2851497 , -0.7116769 ,
        0.03019357, -0.3906174 , -0.3659378 ,  0.00144967,  0.38653368,
       -0.23485576, -0.34210858, -0.20363171,  0.34403124,  0.41653848,
        0.06825578, -0.48287377, -0.09530356,  0.11917412, -0.07616127,
        0.1285845 ,  0.23645061, -0.26507956, -0.5528298 ,  0.3363261 ,
        0.13066311,  0.15565126,  0.05150807, -0.22070867, -0.47490087,
        0.48308465,  0.30766615,  0.2702157 , -0.54587233,  0.73

E é possível até mesmo verificar a similaridade dessa palavra com outras, da seguinte forma:

In [217]:
print(model.wv.most_similar("call", topn=10))

[('claim', 0.9988358020782471), ('prize', 0.9984167814254761), ('cash', 0.9982411861419678), ('guaranteed', 0.9979369640350342), ('stop', 0.9976924657821655), ('<NUMBER>p', 0.9976699948310852), ('reply', 0.9975823760032654), ('free', 0.9975513815879822), ('code', 0.9975404143333435), ('landline', 0.9972397089004517)]


Dessa forma, agora que foi visto tudo que podemos fazer usando o modelo de Word Embedding, podemos usar ele para transformar todo o nosso dataset para um formato que será possível usar ele para o treinamento.

Vamos começar definindo uma função que irá transformar uma sentença inteira um vetor de números.

> A função abaixo é uma ligeira adaptação do código do professor Renato, e pode ser encontrado [nesse link.](https://facens.instructure.com/courses/7531/files/1356091?module_item_id=269232) 

In [107]:
def getDocvector(sentence):
  sentenceValues = []

  for word in sentence:
    # É interessante notar o uso do try aqui dentro,
    # ele se dá porque, durante a criação do modelo,
    # nós especificamos que ele só gere o modelo com
    # palavras que aparecem ao menos 2 vezes. Dessa 
    # forma, palavras com frequencia menor que isso
    # são simplesmente ignoradas, e ai podem causar 
    # que ao tentar acessar elas, ocorra uma exceção
    # porque ela não existe no modelo.
    try:
      sentenceValues.append(model.wv[word])
    except:
      pass

  if len(sentenceValues) > 0:
    return np.mean(sentenceValues, axis = 0)

  return np.zeros(model.wv.vector_size)

print('Testando com uma frase qualquer')
print(getDocvector(sentences[0]))

Testando com uma frase qualquer
[-0.11672595  0.2745955   0.04441339  0.07579166  0.12465201 -0.42670077
  0.01132507  0.6039571  -0.18153174 -0.21091565 -0.07003578 -0.3805226
 -0.01137356  0.15302777  0.13826711 -0.17138228  0.1075444  -0.27268615
 -0.04328139 -0.56410366  0.13783337  0.10297281  0.04997312 -0.08009892
 -0.03188413 -0.01270112 -0.22121951 -0.11836058 -0.23573098  0.01699887
  0.32140818 -0.01367211  0.16152841 -0.22892676 -0.11343078  0.33668715
  0.04975301 -0.24453422 -0.12655662 -0.44680744  0.04213244 -0.22293894
 -0.17945787 -0.00740952  0.2335294  -0.1692181  -0.1897829  -0.13823754
  0.17744662  0.23862854  0.11006291 -0.23513068 -0.09061924  0.04901331
 -0.09417224  0.06691025  0.12474675 -0.13034004 -0.35810268  0.15131949
  0.0208843   0.10032397  0.05801168 -0.09565707 -0.30134565  0.28446952
  0.1835546   0.13147758 -0.33692914  0.3735667  -0.06007675  0.08685918
  0.25312957 -0.07398761  0.3727713   0.11910027 -0.00283908 -0.07339457
 -0.2383779   0.0820

Com a função acima, podemos agora transformar todas as nossas mensagens do modelo para um vetores de valores, assim como, separar nossos valores de classificação e valores de treino e teste, da seguinte forma:

- x = Valores que podem ser usados para descrever se é spam ou não
- y = A informação se é realmente um spam

In [194]:
y = data_set_processed["Category"].to_numpy()
x = []

for sentence in data_set_processed["Message"]:
  x.append(getDocvector(sentence.split()))
  
x = np.array(x)

print("Shape do vetor de classe: ", y.shape)
print("Shape do vetor de valores: ", x.shape)

Shape do vetor de classe:  (5157,)
Shape do vetor de valores:  (5157, 100)


## Separando os Dados

Bom, após ter o modelo para poder transformar os textos em números, podemos agora começar a treinar o nosso modelo, mas antes, precisamos separar os dados que serão usados no treino.

Vamos separar os valores em:

- x_train = Valores de treino
- y_train = A classe se diz se é spam ou não de treino
- x_test = Valores para testar o modelo após o treino
- y_test = A classe dizendo se é spam ou não de teste para testar o modelo após o treino

In [195]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1000)

Além disso, vamos nos certificar que a tipagem dos valores estejam corretos, ou seja, todos sejam do tipo float.

In [198]:
x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

Por fim, temos as seguintes métricas sobre os nossos dados de teste e treino.

In [201]:
print("Qntd. de amostras de treino: ", x_train.shape[0])
print("Qntd de amostras de teste: ", x_test.shape[0])

print("\nQtd. de dados de cada classe (treinamento)")
print("\tClasse ham: ", len(y_train[y_train == 'ham']))
print("\tClasse spam: ", len(y_train[y_train == 'spam']))

print("\nQtd. de dados de cada classe (teste)")
print("\tClasse ham: ", len(y_test[y_test == 'ham']))
print("\tClasse spam: ", len(y_test[y_test == 'spam']))

Qntd. de amostras de treino:  3609
Qntd de amostras de teste:  1548

Qtd. de dados de cada classe (treinamento)
	Classe ham:  3164
	Classe spam:  445

Qtd. de dados de cada classe (teste)
	Classe ham:  1352
	Classe spam:  196


## Treinamento

Agora, com os dados separados, vamos começar o treinamento usando o método de Árvore de Decisão.

Primeiro, importamos o método que iremos usar.

In [203]:
import sklearn
from sklearn.tree import DecisionTreeClassifier

Depois, criamos o nosso modelo da seguinte forma:

In [204]:
decisionTree = DecisionTreeClassifier(criterion = "entropy", random_state=0)

E ai, já podemos começar o treinamento realmente do modelo.

In [205]:
decisionTree.fit(x_train, y_train) 

DecisionTreeClassifier(criterion='entropy', random_state=0)

E com o modelo treinado, podemos começar a ver as métricas de acurácia do nosso modelo tentando predizer os valores de teste que separamos na etapa anterior.

In [206]:
y_pred = decisionTree.predict(x_test) 

Com os valores que foram classificados acima, podemos ver qual foi a sua performance usando o `sklearn` da seguinte forma:

In [212]:
results = sklearn.metrics.classification_report(y_test, y_pred, target_names=['ham', 'spam'])

print(results)

              precision    recall  f1-score   support

         ham       0.96      0.96      0.96      1352
        spam       0.71      0.73      0.72       196

    accuracy                           0.93      1548
   macro avg       0.83      0.84      0.84      1548
weighted avg       0.93      0.93      0.93      1548



Com a tabela acima, podemos tirar algumas conclusões como: 

A precisão para identificar mensagens verdadeiras é maior do que a precisão do que identificar mensagens de spam. E isso não necessariamente é ruim porque é melhor errar mais mensagens de spam do que mensagens verdadeiras.

Dessa forma, no pior dos casos ainda vemos o spam e ignoramos mas com a mensagem real indo parar no spam, é mais dificil de ir lá olhar.

E além disso, podemos checar a nossa acurácia com mais precisão tanto dos valores de teste quanto dos valores de treino da seguinte forma:

In [219]:
accuracyTest = accuracy_score(y_test, decisionTree.predict(x_test))
accuracyTrain = accuracy_score(y_train, decisionTree.predict(x_train))

print("Acurácia sobre os valores de treino: ", accuracyTest)
print("Acurácia sobre os valores de teste: ", accuracyTrain)

Acurácia sobre os valores de treino:  0.9276485788113695
Acurácia sobre os valores de teste:  1.0


E aqui vemos, temos, assim como mostrado no relatório anterior, certa de 93% de acurácia sobre os valores de teste e 100% para os valores de treino.